# Parquet Modular Encryption Example
## Key Management by Application

Configure IBM Analytics Engine accoring to https://cloud.ibm.com/docs/AnalyticsEngine?topic=AnalyticsEngine-parquet-encryption and choose it as the engine for the notebook.

Define path to the folder with encrypted parquet files:

In [7]:
val encryptedParquetFullName = "/tmp/bloodtests.parquet.encrypted"

encryptedParquetFullName = /tmp/bloodtests.parquet.encrypted


/tmp/bloodtests.parquet.encrypted

Setup keys for Parquet Encryption:

In [8]:
sc.hadoopConfiguration.set("encryption.key.list",
      "key1: AAECAwQFBgcICQoLDA0ODw==, key2: AAECAAECAAECAAECAAECAA==")

Generate data:

In [9]:
val dataRange = (1 to 40).toList
val bloodTestList = dataRange.map(i => (i, (i * 2)))

dataRange = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40)
bloodTestList = List((1,2), (2,4), (3,6), (4,8), (5,10), (6,12), (7,14), (8,16), (9,18), (10,20), (11,22), (12,24), (13,26), (14,28), (15,30), (16,32), (17,34), (18,36), (19,38), (20,40), (21,42), (22,44), (23,46), (24,48), (25,50), (26,52), (27,54), (28,56), (29,58), (30,60), (31,62), (32,64), (33,66), (34,68), (35,70), (36,72), (37,74), (38,76), (39,78), (40,80))


List((1,2), (2,4), (3,6), (4,8), (5,10), (6,12), (7,14), (8,16), (9,18), (10,20), (11,22), (12,24), (13,26), (14,28), (15,30), (16,32), (17,34), (18,36), (19,38), (20,40), (21,42), (22,44), (23,46), (24,48), (25,50), (26,52), (27,54), (28,56), (29,58), (30,60), (31,62), (32,64), (33,66), (34,68), (35,70), (36,72), (37,74), (38,76), (39,78), (40,80))

Write encrypted data:

In [10]:
bloodTestList.toDF("id", "value").write
        // Configure which columns to encrypt with which keys
      .option("encryption.column.keys", "key1: id")
      .option("encryption.footer.key", "key2")
      .mode("overwrite").parquet(encryptedParquetFullName)

Read encrypted data:

In [11]:
val encrypedDataDF = spark.read.parquet(encryptedParquetFullName)
encrypedDataDF.createOrReplaceTempView("bloodtests")
val queryResult = spark.sql("SELECT id, value FROM bloodtests")
queryResult.show(10, false)

+---+-----+
|id |value|
+---+-----+
|1  |2    |
|2  |4    |
|3  |6    |
|4  |8    |
|5  |10   |
|6  |12   |
|7  |14   |
|8  |16   |
|9  |18   |
|10 |20   |
+---+-----+
only showing top 10 rows



encrypedDataDF = [id: int, value: int]
queryResult = [id: int, value: int]


[id: int, value: int]

Make sure files were written with parquet encryption (in encrypted footer mode):

In [12]:
import scala.sys.process._
val parquetPartitionFile = Seq("hdfs", "dfs", "-ls", "-S", "-C", encryptedParquetFullName).!!.split("\\s+")(0)
Seq("hdfs", "dfs", "-tail", parquetPartitionFile) #| "tail -c 4" !

PARE

parquetPartitionFile = /tmp/bloodtests.parquet.encrypted/part-00000-87815d7c-fb42-45c2-a568-398aba00b608-c000.snappy.parquet


0

The output of the cell above should be <em>"PARE"</em>